## Part 1

In [1]:
#Import Libs
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [2]:
#Define the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
#Access the Data from Wiki and save as DataFrame
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
tables = soup.select('table[class = "wikitable sortable"]')
df_list = []
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))
    pass
df_wiki = pd.concat(df_list)
df_wiki.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Clean the DataFrame
df_data=df_wiki[~df_wiki['Borough'].isin(['Not assigned'])].reset_index().drop(columns='index')
df_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
#Print the number of rows of DataFrame
df_data.shape

(103, 3)

## Part 2

In [ ]:
#Get Access to the Geo Data
geo_data_url = 'http://cocl.us/Geospatial_data'
df_geo_data = pd.read_csv(geo_data_url)
df_geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
#Combine the DataFrame
df_data_total = pd.merge(df_data,df_geo_data,how='inner',on='Postal Code')
df_data_total.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3

In [ ]:
#Visualize the map
!pip install folium
import folium

In [ ]:
#Visualize the Canada Map
map_canada = folium.Map(location=[43.8, -79.4], zoom_start=11)
map_canada

In [12]:
#Mark on the MAP
incidents = folium.map.FeatureGroup()
for lat, lng, in zip(df_data_total.Latitude, df_data_total.Longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
map_canada.add_child(incidents)

In [13]:
#It seems like that neighborhoods distribution in Toronto is relatively even